In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-song/data.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import classification_report
import numpy as np

In [4]:
df = pd.read_csv('/kaggle/input/text-song/data.csv')

In [5]:
X = df['Текст']
y = df[['Грусть/Печаль/Страдания', 'Оптимизм/Веселье/Счастье', 'Новый год/Рождество', 
        'Любовь/Романтика', 'Патриотическая песня', 'Лето', 'Осень']]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [6]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=7)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=512)


train_labels = torch.tensor(y_train.values, dtype=torch.float).to(device)
val_labels = torch.tensor(y_val.values, dtype=torch.float).to(device)
test_labels = torch.tensor(y_test.values, dtype=torch.float).to(device)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:


train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']).to(device), 
                              torch.tensor(train_encodings['attention_mask']).to(device), 
                              train_labels)
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']).to(device), 
                            torch.tensor(val_encodings['attention_mask']).to(device), 
                            val_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']).to(device), 
                             torch.tensor(test_encodings['attention_mask']).to(device), 
                             test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [8]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)


loss_fn = torch.nn.BCEWithLogitsLoss().to(device)

In [ ]:



model.train()

for epoch in range(3):  
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        # Получение данных
        input_ids, attention_mask, labels = batch
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
              
        loss = loss_fn(logits, labels)
        
        loss.backward()
        
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch + 1} - Loss: {total_loss / len(train_loader)}")
    
    # Валидация
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fn(logits, labels)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss / len(val_loader)}")

100%|██████████| 402/402 [02:59<00:00,  2.24it/s]


Epoch 1 - Loss: 0.10122904870010431
Validation Loss: 0.1046875602105397


100%|██████████| 402/402 [02:59<00:00,  2.24it/s]


Epoch 2 - Loss: 0.09271155101993117
Validation Loss: 0.10457797824865443


 53%|█████▎    | 214/402 [01:35<01:24,  2.24it/s]

In [12]:


model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.sigmoid(logits).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


predictions = (np.array(predictions) > 0.5).astype(int)


print(classification_report(true_labels, predictions, target_names=y.columns))

                          precision    recall  f1-score   support

 Грусть/Печаль/Страдания       0.85      0.91      0.88       247
Оптимизм/Веселье/Счастье       0.80      0.87      0.83       180
     Новый год/Рождество       0.64      0.58      0.61        12
        Любовь/Романтика       0.89      0.97      0.93        93
    Патриотическая песня       0.00      0.00      0.00         5
                    Лето       0.92      0.69      0.79       120
                   Осень       0.62      0.91      0.74        44

               micro avg       0.83      0.86      0.84       701
               macro avg       0.67      0.70      0.68       701
            weighted avg       0.83      0.86      0.84       701
             samples avg       0.83      0.86      0.84       701



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
def make_contiguous(model):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

In [17]:
make_contiguous(model)
model.save_pretrained('/kaggle/working/')
tokenizer.save_pretrained('/kaggle/working/')

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/vocab.txt',
 '/kaggle/working/added_tokens.json')